<a href="https://colab.research.google.com/github/Creeperwk/IMDB-SQL-Project/blob/main/IMDB_data_SQL_practice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Preparations**

In [3]:
# Import the packages
import numpy as np
import pandas as pd
import sqlite3 as sql

In [14]:
# Connect the database
Database = "/content/imdb-project-sql/movies.sqlite"
db = sql.connect(Database)
# Create a cursor object
cur = db.cursor()

In [18]:
#Get movie column names
cur.execute("PRAGMA table_info(movies)")
movies = cur.fetchall()

movies_column_names = []
for column_info in movies:
    column_name = column_info[1]
    movies_column_names.append(column_name)
print(movies_column_names)


['id', 'original_title', 'budget', 'popularity', 'release_date', 'revenue', 'title', 'vote_average', 'vote_count', 'overview', 'tagline', 'uid', 'director_id']


In [17]:
# Get director column names
cur.execute("PRAGMA table_info(directors)")
directors = cur.fetchall()

directors_column_names = []
for column_info in directors:
    column_name = column_info[1]
    directors_column_names.append(column_name)
print(directors_column_names)

['name', 'id', 'gender', 'uid', 'department']

**Question 1**: Can you get all data about movies?

In [47]:
# Fetch the movie data
cur.execute("SELECT * FROM movies")
movies = cur.fetchall()
movies = pd.DataFrame(movies, columns=movies_column_names)
movies.head(3)

,id,original_title,budget,popularity,release_date,revenue,title,vote_average,vote_count,overview,tagline,uid,director_id
0,43597,Avatar,237000000,150,2009-12-10,2787965087,Avatar,7.2,11800,"In the 22nd century, a paraplegic Marine is di...",Enter the World of Pandora.,19995,4762
1,43598,Pirates of the Caribbean: At World's End,300000000,139,2007-05-19,961000000,Pirates of the Caribbean: At World's End,6.9,4500,"Captain Barbossa, long believed to be dead, ha...","At the end of the world, the adventure begins.",285,4763
2,43599,Spectre,245000000,107,2015-10-26,880674609,Spectre,6.3,4466,A cryptic message from Bond’s past sends him o...,A Plan No One Escapes,206647,4764


In [ ]:
# Data summary
movies.info()

**Question 2**: How do you get all data about directors?

In [34]:
# Fetch the director data
cur.execute("SELECT * FROM directors")
directors = cur.fetchall()
directors = pd.DataFrame(directors, columns=directors_column_names)
directors.head(3)

,name,id,gender,uid,department
0,James Cameron,4762,2,2710,Directing
1,Gore Verbinski,4763,2,1704,Directing
2,Sam Mendes,4764,2,39,Directing


In [33]:
# Data summary
directors.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2349 entries, 0 to 2348
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   name        2349 non-null   object
 1   id          2349 non-null   int64 
 2   gender      2349 non-null   int64 
 3   uid         2349 non-null   int64 
 4   department  2349 non-null   object
dtypes: int64(3), object(2)
memory usage: 91.9+ KB


**Question 3**: Check how many movies are present in iMDB.

In [ ]:
cur.execute("SELECT COUNT(*) FROM movies")
movies_count = cur.fetchall()

print("The iMDB database contains information of "+str(movies_count[0][0])+" movies.")

**Question 4**: Find these 3 directors: James Cameron ; Luc Besson ; John Woo

In [ ]:
cur.execute("SELECT * FROM directors WHERE name IN ('James Cameron','Luc Besson', 'John Woo')")
selected_directors = cur.fetchall()
selected_directors = pd.DataFrame(selected_directors, columns=directors_column_names)
selected_directors

**Question 5**: Find all directors with name starting with Steven.

In [ ]:
cur.execute("SELECT * FROM directors WHERE name LIKE 'Steven%'")
Steven_directors = cur.fetchall()
Steven_directors = pd.DataFrame(Steven_directors, columns=directors_column_names)
Steven_directors

**Question 6**: Count female directors.

In [ ]:
# We know that there are three gender types from data and '1' represent female
cur.execute("SELECT DISTINCT gender FROM directors")
gender_type = cur.fetchall()
gender_type

In [ ]:
cur.execute("SELECT count(*) FROM directors WHERE gender == '1' ")
female_number = cur.fetchall()
print('There are ' + str(female_number[0][0]) + ' female directors in the database')

**Question 7**: Find the name of the 10th first women directors?

In [ ]:
# Ranked female directors by uid
cur.execute("SELECT name FROM directors WHERE gender == '1' ORDER BY uid ASC ")
female_directors = cur.fetchall()
female_directors[0:10]

**Question 8**: What are the 3 most popular movies?


In [ ]:
# Rank the movies by popularity and fetch the top three
cur.execute("SELECT * FROM movies ORDER BY popularity DESC LIMIT 3")
popular_movies = cur.fetchall()
print("The 3 most popular movies are:\n")
for movie in popular_movies:
    print(movie[1])

**Question 9**: What are the 3 most bankable movies?

In [ ]:
# Rank the movies by (total revenue - budget) and fetch the top three
cur.execute("SELECT * FROM movies ORDER BY (revenue - budget) DESC LIMIT 3")
bankable_movies = cur.fetchall()
print("The 3 most bankable movies are:\n")
for movie in bankable_movies:
    print(movie[1])

**Question 10**: What is the most awarded average vote since the January 1st, 2000?

In [70]:
# Select the max value for voted average since 2000-1-1
cur.execute("SELECT MAX(vote_average) FROM movies WHERE release_date >= '2000-01-01'")
most_voted_average = cur.fetchall()
print("The most awarded average vote since the January 1st 2000 is "+str(most_voted_average[0][0])+".")

The most awarded average vote since the January 1st 2000 is 9.5.


**Question 11**: Which movie(s) were directed by Brenda Chapman?

In [71]:
# Select the movie which have the same director id as the one from directors database
cur.execute("""SELECT original_title
              FROM movies JOIN directors ON directors.id = movies.director_id
              WHERE directors.name = 'Brenda Chapman'""")
movies_Brenda_Chapman = cur.fetchall()
print("Movies directed by Brenda Chapman: "+str(movies_Brenda_Chapman[0][0]))

Movies directed by Brenda Chapman: Brave


**Question 12**: Whose director made the most movies?

In [78]:
# Rank the director id by the number of movies and select the name from the directors database
cur.execute(""" SELECT name, COUNT(director_id)
                FROM movies JOIN directors
                ON directors.id = movies.director_id
                GROUP BY director_id
                ORDER BY COUNT(director_id)
                DESC LIMIT 1"""
)
most_movie_director = cur.fetchall()
print(str(most_movie_director[0][0])+ "made the most movies, which is " +str(most_movie_director[0][1]) )

Steven Spielbergmade the most movies, which is27


**Question 13**: Whose director is the most bankable?

In [82]:
cur.execute(""" SELECT name, (revenue - budget)
                FROM movies JOIN directors
                ON directors.id = movies.director_id
                GROUP BY director_id
                ORDER BY SUM(revenue - budget)
                DESC LIMIT 1""")

most_bankable_director = cur.fetchall()
print(str(most_bankable_director[0][0])+ "made the most money from movies, which the total profit is " + str(most_bankable_director[0][1]) + " dollars")

Steven Spielbergmade the most money from movies, which the total profit is 601636033 dollars
